
# Ekstraksi Metadata dari Putusan Mahkamah Agung
Kode ini bertujuan untuk mengumpulkan metadata dari putusan yang diterbitkan di situs Mahkamah Agung. Metadata ini termasuk informasi tentang terdakwa, penuntut umum, hakim, dan detail lainnya dari setiap kasus.


## Import Library
Beberapa library digunakan dalam skrip ini untuk membantu pengolahan data dan interaksi dengan web:
- `requests`: Untuk mengirim permintaan HTTP ke situs web.
- `BeautifulSoup` dari `bs4`: Untuk parsing dokumen HTML dan ekstraksi data.
- `re`: Untuk melakukan operasi regular expression, yang digunakan untuk mencari dan manipulasi string.
- `csv`: Untuk membaca dan menulis file CSV.
- `os`: Untuk interaksi dengan sistem operasi.
- `time` dan `warnings`: Untuk mengukur waktu eksekusi dan mengatur peringatan.



In [6]:
import requests
import urllib.request
import time
import os
import warnings
import re

from bs4 import BeautifulSoup

import csv
import os.path

## Fungsi generateFileCSV
Fungsi ini bertanggung jawab untuk menciptakan dan menulis ke file CSV:
- Menerima list hasil data dan nama file sebagai parameter.
- Mengecek apakah file sudah ada; jika tidak, file baru dibuat dan header ditulis.
- Menulis data ke dalam file CSV dan menutup file setelah operasi selesai.


In [7]:
def generateFileCSV(listHasil,csvName1):
	csvName = csvName1

	if os.path.exists(csvName):
		f = open(csvName, 'a', newline='\n')
		print(f)
		print("ada")
		w = csv.writer(f)

	else:
		f = open(csvName, 'w', newline='\n')
		print(f)
		print("tidak ada")
		w = csv.writer(f)
		w.writerow(("terdakwa", "penuntut_umum", "nomor", "tingkat_proses", "klasifikasi", "kata_kunci", "tahun", "tanggal_register",
              "lembaga_peradilan", "jenis_lembaga_peradilan", "hakim_ketua", "hakim_anggota", "panitera", "amar",
              "amar_lainnya", "catatan_amar", "tanggal_musyawarah", "tanggal_dibacakan", "kaidah", "abstrak", "url"))

	# menulis file csv
	for s in listHasil:
		w.writerow(s)

	f.close()
	berhasil = "\nCreate Csv file Berhasil\n"
	return berhasil


## Fungsi generateMeta
Fungsi ini mengambil URL sebagai input dan mengumpulkan berbagai metadata dari halaman web:
- Mengirim permintaan HTTP ke URL yang diberikan.
- Menggunakan `BeautifulSoup` untuk parsing HTML.
- Menggunakan ekspresi reguler (`re`) untuk mengekstrak informasi tertentu seperti nama terdakwa dan penuntut umum.
- Menyimpan informasi yang diekstraksi ke dalam list yang kemudian direturn.


In [8]:
def generateMeta(urlMeta):

    url = str(urlMeta).strip()
    #url = 'https://putusan3.mahkamahagung.go.id/direktori/putusan/zaedca28e81ec25e8cb4313634373336.html'
    response = requests.get(url, verify=False)
    #print(url)
    print(response)

    soup = BeautifulSoup(response.text, 'html.parser')
    cleanTags = re.compile('<.*?>')

    listMetaHead = ["terdakwa", "penuntut_umum", "nomor", "tingkat_proses", "klasifikasi", "kata_kunci", "tahun", "tanggal_register",
              "lembaga_peradilan", "jenis_lembaga_peradilan", "hakim_ketua", "hakim_anggota", "panitera", "amar",
              "amar_lainnya", "catatan_amar", "tanggal_musyawarah", "tanggal_dibacakan", "kaidah", "abstrak", "url"]

    # initialisasi
    listMeta = []
    #for i in range(len(listMetaHead)):
        #listMeta.append("")

    rowsMETA1 = soup.find("ul", {"class": "portfolio-meta nobottommargin"}).find("table").findAll("tr")
    rowsMETA2 = soup.findAll("ul", {"class": "portfolio-meta nobottommargin"})

    #print('-------------------------')
    for row in rowsMETA1:
        coll = row.findAll("td")

        cleantext2 =''
        cleantext1 =''

        if len(coll) > 1:
            cleantext2 = (re.sub(cleanTags, '', str(coll[1]))).strip()
            listMeta.append(cleantext2.replace('\n',' '))
            #print(cleantext2.replace('\n',' '))

        else:
            cleantext1 = (re.sub(cleanTags, ' ', str(coll[0]))).strip()
            # untuk putusan pidana akan muncul terdakwa dan penuntut umum pada meta
            # sedangkan untuk putusan selain pidana tidak muncul

            pidorpdt = re.search( r'(.*)/Pdt.(.*)',str(cleantext1), re.M|re.I)

            # check dokumen putusannya pidana atau perdata
            if pidorpdt == None:
                entTerdakwah = re.search( r'(.*)Terdakwa:(.*)',str(cleantext1), re.M|re.I)
                entPenuntut = re.search( r'Penuntut Umum:(.*)Terdakwa:',str(cleantext1), re.M|re.I)
            else:
                entTerdakwah = re.search( r'(.*)Tergugat:(.*)',str(cleantext1), re.M|re.I)
                entPenuntut = re.search( r'Penggugat:(.*)Tergugat:',str(cleantext1), re.M|re.I)

            if entTerdakwah == None:
                listMeta.append("")
            else:
                listMeta.append(entTerdakwah.group(2))

            if entPenuntut == None:
                listMeta.append("")
            else:
                listMeta.append(entPenuntut.group(1))

            #print(coll[0])
            #print(entTerdakwah.group(2))
            #print(entPenuntut.group(1))

    urlDL = rowsMETA2[1].findAll("li")
    urlDLStr = str(urlDL[4])
    listMeta.append(urlDLStr[urlDLStr.find("https"):urlDLStr.find('">')])
    #print(urlDLStr[urlDLStr.find("https"):urlDLStr.find('">')])
    #print(listMeta)

    return listMeta


## Fungsi main
Fungsi ini mengoordinasikan eksekusi keseluruhan:
- Menyiapkan variabel dan membuka file yang berisi daftar URL.
- Membaca setiap URL dan memanggil `generateMeta` untuk mengumpulkan metadata.
- Hasilnya disimpan dan ditulis ke file CSV menggunakan `generateFileCSV`.
- Menghitung waktu eksekusi dan menutup file yang dibuka.


In [9]:
def main():

    warnings.filterwarnings('ignore')

    #folderListURL = pathFile
    fileListURL = "hasilListURLPage.txt"
    fileMetaCSV = "metaPidanaUmumKediri.csv"
    listHasil =[]

    startTime = time.time()
    openfileListURL = open(fileListURL, "r", encoding='UTF8')
    bacaListURL = openfileListURL.readlines()

    i = 1
    for barisURL in bacaListURL:
        try:
            #print(str(barisURL))
            hasil = generateMeta(str(barisURL))
            listHasil.append(hasil)
            print("======= ROW HASIL =======",i)
            #print(hasil)
        except Exception as e:
            print(f"Error Get Meta Inf, {e}")
        i=i+1

    createFile = generateFileCSV(listHasil,fileMetaCSV)

    openfileListURL.close()
    endTime = time.time()
    #print(listHasil)
    print('Time Processing : ', endTime-startTime, ' Second')


## Eksekusi
Blok `if __name__ == "__main__":` memastikan bahwa skrip hanya dijalankan ketika file dijalankan sebagai program utama dan bukan saat diimpor sebagai modul. Ini berguna untuk mencegah eksekusi fungsi `main` secara tidak sengaja.


In [10]:
if __name__ == "__main__":
    main()

<Response [200]>
======= ROW HASIL ======= 1
<Response [200]>
======= ROW HASIL ======= 2
<Response [200]>
======= ROW HASIL ======= 3
<Response [200]>
======= ROW HASIL ======= 4
<Response [200]>
======= ROW HASIL ======= 5
<Response [200]>
======= ROW HASIL ======= 6
<Response [200]>
======= ROW HASIL ======= 7
<Response [200]>
======= ROW HASIL ======= 8
<Response [200]>
======= ROW HASIL ======= 9
<Response [200]>
======= ROW HASIL ======= 10
<Response [200]>
======= ROW HASIL ======= 11
<Response [200]>
======= ROW HASIL ======= 12
<Response [200]>
======= ROW HASIL ======= 13
<Response [200]>
======= ROW HASIL ======= 14
<Response [200]>
======= ROW HASIL ======= 15
<Response [200]>
======= ROW HASIL ======= 16
<Response [200]>
======= ROW HASIL ======= 17
<Response [200]>
======= ROW HASIL ======= 18
<Response [200]>
======= ROW HASIL ======= 19
<Response [200]>
======= ROW HASIL ======= 20
<Response [200]>
======= ROW HASIL ======= 21
<Response [200]>
======= ROW HASIL ======= 